In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

In [3]:
system_name = "FRECON"
location = "Thailand"
base_url = "https://pvoutput.org/list.jsp?p={}&id=51787&sid=47085&gs=0&v=0&o=date&d=desc"
pages_to_scrape = 5

In [4]:
options = Options()
options.binary_location = r"C:\Program Files\chrome for testing\chrome-win64\chrome.exe"
options.add_argument("--headless=new")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36")
service = Service(executable_path=r"C:\Program Files\chrome for testing\chrome-win64\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)

In [5]:
all_data = []

for page in range(pages_to_scrape):
    url = base_url.format(page)
    print(f"📄 Scraping page {page + 1} from: {url}")
    
    try:
        driver.get(url)
        time.sleep(6)
        soup = BeautifulSoup(driver.page_source, "html.parser")
        table = soup.find("table", id="tbl_main")

        if not table:
            print(f"⚠️ No table found on page {page + 1}. Retrying once...")
            time.sleep(5)
            driver.get(url)
            soup = BeautifulSoup(driver.page_source, "html.parser")
            table = soup.find("table", id="tbl_main")

        if table:
            rows = table.find_all("tr")[1:]
            for row in rows:
                cols = [td.get_text(strip=True).replace(',', '') for td in row.find_all("td")]
                if len(cols) >= 9:
                    cols.append(system_name)
                    cols.append(location)
                    all_data.append(cols)
        else:
            print(f"❌ Failed again: No table found on page {page + 1}")
    except Exception as e:
        print(f"🚨 Error scraping page {page + 1}: {e}")

driver.quit()

📄 Scraping page 1 from: https://pvoutput.org/list.jsp?p=0&id=51787&sid=47085&gs=0&v=0&o=date&d=desc
📄 Scraping page 2 from: https://pvoutput.org/list.jsp?p=1&id=51787&sid=47085&gs=0&v=0&o=date&d=desc
📄 Scraping page 3 from: https://pvoutput.org/list.jsp?p=2&id=51787&sid=47085&gs=0&v=0&o=date&d=desc
📄 Scraping page 4 from: https://pvoutput.org/list.jsp?p=3&id=51787&sid=47085&gs=0&v=0&o=date&d=desc
📄 Scraping page 5 from: https://pvoutput.org/list.jsp?p=4&id=51787&sid=47085&gs=0&v=0&o=date&d=desc


In [6]:
header = [
    "Date", "Generated", "Efficiency", "Exported", "Peak Power",
    "Peak Time", "Conditions", "Temperature", "Comments",
    "System Name", "Location"
]

In [7]:
df = pd.DataFrame(all_data, columns=header)
df.drop(columns=["Exported", "Temperature", "Comments", "Peak Power", "Peak Time", "Conditions"], inplace=True)
print(df.head())

       Date   Generated   Efficiency System Name  Location
0  31/03/25    0.000kWh  0.000kWh/kW      FRECON  Thailand
1  30/03/25    0.000kWh  0.000kWh/kW      FRECON  Thailand
2  29/03/25  552.301kWh  3.607kWh/kW      FRECON  Thailand
3  28/03/25  600.951kWh  3.924kWh/kW      FRECON  Thailand
4  27/03/25  536.289kWh  3.502kWh/kW      FRECON  Thailand


In [8]:
page6 = pd.read_csv("FRECON_page6.csv")
page7 = pd.read_csv("FRECON_page7.csv")
page8 = pd.read_csv("FRECON_page8.csv")
page9 = pd.read_csv("FRECON_page9.csv")
page10 = pd.read_csv("FRECON_page10.csv")

In [9]:
page6["System Name"] = "FRECON"
page6["Location"] = "Thailand"
page7["System Name"] = "FRECON"
page7["Location"] = "Thailand"
page8["System Name"] = "FRECON"
page8["Location"] = "Thailand"
page9["System Name"] = "FRECON"
page9["Location"] = "Thailand"
page10["System Name"] = "FRECON"
page10["Location"] = "Thailand"

In [10]:
master_df = pd.concat([df, page6, page7, page8, page9, page10], ignore_index=True)

In [11]:
master_df

,Date,Generated,Efficiency,System Name,Location
0,31/03/25,0.000kWh,0.000kWh/kW,FRECON,Thailand
1,30/03/25,0.000kWh,0.000kWh/kW,FRECON,Thailand
2,29/03/25,552.301kWh,3.607kWh/kW,FRECON,Thailand
3,28/03/25,600.951kWh,3.924kWh/kW,FRECON,Thailand
4,27/03/25,536.289kWh,3.502kWh/kW,FRECON,Thailand
...,...,...,...,...,...
395,2/2/2024,492.147,3.213706,FRECON,Thailand
396,2/1/2024,451.547,2.94859,FRECON,Thailand
397,1/31/2024,401.362,2.620883,FRECON,Thailand
398,1/30/2024,361.826,2.362714,FRECON,Thailand


In [12]:
master_df["System Size (kW)"] = 153.140

In [13]:
master_df.to_csv("daily_FRECON_master.csv", index=False)